## Creación del conjunto de datos.

In [1]:
## El conjunto de entrenamiento debe de cumplir: 
# and,0,0,0
# and,0,1,0
# and,1,0,0
# and,1,1,1
# or,0,0,0
# or,0,1,1
# or,1,0,1
# or,1,1,1
# xor,0,0,0
# xor,0,1,1
# xor,1,0,1
# xor,1,1,0

In [2]:
import torch
from torch import nn 
import matplotlib.pyplot as plt
import numpy as np
import random

In [3]:
def code_and(a,b):
    return a and b

def code_or(a,b):
    return a or b

def code_xor(a,b):
    return a ^ b


In [4]:
def generate_data(gate, code, samples):
    data = []
    labels = []
    for _ in range(samples):        
        x1 = np.random.randint(0, 2)
        x2 = np.random.randint(0, 2)        
        y = code(x1, x2)        
        x = np.array([gate, x1, x2])        
        data.append(x)
        labels.append(y)        
    
    data = np.array(data)
    labels = np.array(labels)
    
    # Cambiar la forma del array de etiquetas para que tenga dos dimensiones
    labels = np.reshape(labels, (-1, 1))
    
    return data, labels

In [5]:
def create_data(gate_name, code_function, samples, data, labels):    
    _data, _labels = generate_data(gate_name, code_function, samples)    
    data = np.concatenate((data, _data))
    labels = np.concatenate((labels, _labels))
    
    return data, labels

In [6]:
import numpy as np
samples = 5000
# Crear los arrays vacíos con la misma forma que los arrays que se van a generar
data = np.empty((0, 3))
labels = np.empty((0, 1))

# Crear los datos y las etiquetas para la compuerta AND
data, labels = create_data(1, code_and, samples, data, labels)
# Crear los datos y las etiquetas para la compuerta OR
data, labels = create_data(2, code_or, samples, data, labels)
# Crear los datos y las etiquetas para la compuerta XOR
data, labels = create_data(3, code_xor, samples, data, labels)

In [7]:
X = torch.Tensor(data)
y = torch.Tensor(labels)
y = y.squeeze(1)


In [8]:
X[0]

tensor([1., 0., 0.])

In [9]:
y[0]

tensor(0.)

## Creación del conjunto de entrenamiento

In [10]:
# Create train/test split
train_split = int(0.8 * len(data)) # 80% of data used for training set, 20% for testing 
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]

len(X_train), len(y_train), len(X_test), len(y_test)

(12000, 12000, 3000, 3000)

## Creación del modelo

In [11]:
# Standard PyTorch imports
import torch
from torch import nn

# Make device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device


'cuda'

In [12]:
# Create a model with non-linear and linear layers
class GatesModels(nn.Module):
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()
        self.layer_stack = nn.Sequential(        
            nn.Linear(in_features=input_shape, out_features=hidden_units),
            #nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=output_shape),
            #nn.ReLU()
        )
    
    def forward(self, x: torch.Tensor):
        return self.layer_stack(x)

In [13]:

model= GatesModels(3,8,1).to(device)
model

GatesModels(
  (layer_stack): Sequential(
    (0): Linear(in_features=3, out_features=8, bias=True)
    (1): Linear(in_features=8, out_features=1, bias=True)
  )
)

In [14]:
# Create a loss function
# loss_fn = nn.BCELoss() # BCELoss = no sigmoid built-in
loss_fn = nn.BCEWithLogitsLoss() # BCEWithLogitsLoss = sigmoid built-in

# Create an optimizer
optimizer = torch.optim.SGD(params=model.parameters(), 
                            lr=0.5)

In [15]:
# Calculate accuracy (a classification metric)
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100 
    return acc

In [16]:
untrained_preds = model(X_test.to(device))

## Entrenamiento

In [17]:
torch.manual_seed(42)

# Set the number of epochs
epochs = 100

# Put data to target device
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)


# Build training and evaluation loop
for epoch in range(epochs):
    ### Training
    model.train()

    # 1. Forward pass (model outputs raw logits)
    y_logits = model(X_train).squeeze() # squeeze to remove extra `1` dimensions, this won't work unless model and data are on same device 
    y_pred = torch.round(torch.sigmoid(y_logits)) # turn logits -> pred probs -> pred labls
  
    # 2. Calculate loss/accuracy
    # loss = loss_fn(torch.sigmoid(y_logits), # Using nn.BCELoss you need torch.sigmoid()
    #                y_train) 
    loss = loss_fn(y_logits, # Using nn.BCEWithLogitsLoss works with raw logits
                   y_train) 
    acc = accuracy_fn(y_true=y_train, 
                      y_pred=y_pred) 

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backwards
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    ### Testing
    model.eval()
    with torch.inference_mode():
        # 1. Forward pass
        test_logits = model(X_test).squeeze() 
        test_pred = torch.round(torch.sigmoid(test_logits))
        # 2. Caculate loss/accuracy
        test_loss = loss_fn(test_logits,
                            y_test)
        test_acc = accuracy_fn(y_true=y_test,
                               y_pred=test_pred)

    # Print out what's happening every 10 epochs
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Accuracy: {acc:.2f}% | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%")

Epoch: 0 | Loss: 0.67865, Accuracy: 49.86% | Test loss: 0.70886, Test acc: 24.57%
Epoch: 10 | Loss: 0.59774, Accuracy: 71.13% | Test loss: 0.74003, Test acc: 48.50%
Epoch: 20 | Loss: 0.53985, Accuracy: 75.36% | Test loss: 0.79587, Test acc: 73.50%
Epoch: 30 | Loss: 0.49417, Accuracy: 85.30% | Test loss: 0.87354, Test acc: 73.50%
Epoch: 40 | Loss: 0.46394, Accuracy: 95.72% | Test loss: 0.96259, Test acc: 73.50%
Epoch: 50 | Loss: 0.44645, Accuracy: 95.72% | Test loss: 1.04942, Test acc: 73.50%
Epoch: 60 | Loss: 0.43711, Accuracy: 95.72% | Test loss: 1.12444, Test acc: 73.50%
Epoch: 70 | Loss: 0.43231, Accuracy: 95.72% | Test loss: 1.18445, Test acc: 73.50%
Epoch: 80 | Loss: 0.42989, Accuracy: 95.72% | Test loss: 1.23028, Test acc: 73.50%
Epoch: 90 | Loss: 0.42867, Accuracy: 95.72% | Test loss: 1.26437, Test acc: 73.50%


In [18]:
tensor_data = torch.tensor([3.0,0.0,0.0]).to(device)

In [19]:
test_logits = model(tensor_data).squeeze() 
test_pred = torch.round(torch.sigmoid(test_logits))

In [20]:
test_pred

tensor(0., device='cuda:0', grad_fn=<RoundBackward0>)